https://stackoverflow.com/questions/43355044/cumulative-explained-variance-for-pca-in-python

https://stackoverflow.com/questions/32011359/convert-categorical-data-in-pandas-dataframe

https://stackoverflow.com/questions/23294616/how-to-use-scikit-learn-pca-for-features-reduction-and-know-which-features-are-d

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/

## imports

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import copy
import time
from math import sqrt

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.svm import SVR,SVC
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from rgf.sklearn import RGFClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

## Functions

In [27]:
def execution_time_calculator(start_time):
    delta = time.time() - start_time
    print("--- %s seconds ---" % (delta))
    

# def df_clean(df):
#     df["date"] = df["date"].apply(time_to_standard)
#     df = df.drop_duplicates(subset=['date'])
    
#     dropped_df = df.dropna(axis=1).dropna().drop(['unix', 'symbol'], axis=1)
    
    
#     return dropped_df


def plot_train_data(df):
    plt.figure(figsize=(30,10))
    plt.plot(df.index, df['close'])
    
    #plt.title('Price chart (close) ' + str(ticker))
    plt.show()
    return None


def df_clean(df,mode=0,pre_found_age_value=0):
    if mode==0:
        age_mean = df.mean(axis=0)[3]
    elif mode==1:
        age_mean = pre_found_age_value
    
    values = {"Age": age_mean, "Cabin": "-", "Embarked": "-","Pclass":0,"Sex":"male","SibSp":0,"Parch":0,"Ticket":"-","Fare":0}
    df = df.fillna(value=values)
    # df = df[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked", "Survived"]]
    if mode==0:
#         df = df
        df = df[['Pclass', 'Sex', 'Age', "SibSp", "Parch", "Fare", "Cabin", "Embarked", "Survived"]]
    elif mode==1:
        df = df[['Pclass', 'Sex', 'Age', "SibSp", "Parch", "Fare", "Cabin", "Embarked"]]

    return df,age_mean

def df_encoder(df,mode=0,LEs=None):
    if mode==0:
        LE_Sex = LabelEncoder()
        LE_Cabin = LabelEncoder()
        LE_Embarked = LabelEncoder()
    elif mode==1:
        LE_Sex = LEs[0]
        LE_Cabin = LEs[0]
        LE_Embarked = LEs[0]
    
    df['Sex'] = LE_Sex.fit_transform(df['Sex'])
    df['Cabin'] = LE_Cabin.fit_transform(df['Cabin'])
    df['Embarked'] = LE_Embarked.fit_transform(df['Embarked'])
    
    return df, LE_Sex, LE_Cabin, LE_Embarked


def metrics_calculator(real,pred,mode):
    print("********",mode,"********",":")
    metrics_lists=[metrics.f1_score,metrics.accuracy_score,metrics.precision_score,metrics.recall_score,metrics.average_precision_score,metrics.roc_auc_score,metrics.mean_squared_error,metrics.mean_squared_error]
    metric_names=["F1","Accuracy","Precision","Recall","AP","RoC_AuC","MSE"]
    for metric in range(len(metrics_lists)):
        if metric== len(metrics_lists)-1:
            metric_value = metrics_lists[metric](real, pred)
            print("{:20}".format("{}: ".format("RMSE")),sqrt(metric_value))
        else:  
            metric_value = metrics_lists[metric](real, pred)
            print("{:20}".format("{}: ".format(metric_names[metric])),metric_value)
        
    report = classification_report(real, pred)
    print(report)


    
def all_models_test(df,models,models_name,x_train,y_train):

    classifiers = []
    for i in range(len(models)):
        print("--------",models_name[i],"--------")
        clf = models[i].fit(x_train, y_train)
        y_pred_train = clf.predict(x_train)
        metrics_calculator(y_pred_train,y_train,"train")
        classifiers.append(clf)
    return classifiers

## Read and clean train data

In [28]:
df = pd.read_csv("./data/train.csv")#,skiprows=1)#.iloc[::-1].reset_index().drop(["index"], axis=1)

In [29]:
print(df.shape)
df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
# df.info()

In [31]:
df,age_mean = df_clean(df)

C:\Users\Tohid\anaconda\envs\pattern7\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [32]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,male,22.0,1,0,7.2500,-,S,0
1,1,female,38.0,1,0,71.2833,C85,C,1
2,3,female,26.0,0,0,7.9250,-,S,1
3,1,female,35.0,1,0,53.1000,C123,S,1
4,3,male,35.0,0,0,8.0500,-,S,0


In [33]:
# df["Cabin"].unique()

## Encode categorical data

In [34]:
df, LE_Sex, LE_Cabin, LE_Embarked = df_encoder(df)


# pandas encoding - not good when we want to apply on test data

# df['Sex'] = df['Sex'].astype('category')
# df['Cabin'] = df['Cabin'].astype('category')
# df['Embarked'] = df['Embarked'].astype('category')
# OneHotEncoder().fit_transform(df)
# cat_columns = df.select_dtypes(['category']).columns
# df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [35]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,3,1,22.0,1,0,7.2500,0,3,0
1,1,0,38.0,1,0,71.2833,82,1,1
2,3,0,26.0,0,0,7.9250,0,3,1
3,1,0,35.0,1,0,53.1000,56,3,1
4,3,1,35.0,0,0,8.0500,0,3,0


In [36]:
# df.info()

In [37]:
array = df.values
X = array[:,:-1]
Y = array[:,-1]

## PCA

In [38]:
pca = PCA(n_components=6)
fit = pca.fit(X)
# summarize components
explained_variance_ratio=fit.explained_variance_ratio_

print(explained_variance_ratio)
print(len(explained_variance_ratio))
print(sum(explained_variance_ratio))
x_pca_train = pca.transform(X)
y_train = Y
x_train = X

[6.69591682e-01 2.89634969e-01 4.00730257e-02 3.10136191e-04
 1.45071161e-04 1.14935589e-04]
6
0.9998698193599481


## Classifiers

In [39]:
BNB = BernoulliNB()
GNB = GaussianNB()
RF = RandomForestClassifier()
DTree = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
LR = LogisticRegression()
MLP = make_pipeline(MinMaxScaler(), MLPClassifier(random_state=0, shuffle=False))
XGB = xgb.XGBClassifier(eval_metric='mlogloss')
SVC_auto = make_pipeline(MinMaxScaler(), SVC(gamma="auto"))
RGF = RGFClassifier()
Bagging = BaggingClassifier(random_state=0)
estimators = [
   ('xgb', xgb.XGBClassifier(eval_metric='mlogloss')),
   ('lr', LogisticRegression()),
   ('nb', BernoulliNB()),
   ('rgf', RGFClassifier()),
   ('svm', make_pipeline(StandardScaler(), SVC(gamma="auto"))),
   ('mlp', make_pipeline(StandardScaler(), MLPClassifier(random_state=0, shuffle=False))),
]
Voting = VotingClassifier(estimators)
Stacking = StackingClassifier(estimators, LogisticRegression())
custom_RF=RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=6, min_samples_leaf=6, warm_start=True)

models = [BNB,GNB,RF,custom_RF,DTree,KNN,LR,MLP,XGB,SVC_auto,RGF,Bagging,Voting,Stacking]
models_name = ["BNB","GNB","RF","custom_RF","DTree","KNN","LR","MLP","XGB","SVC_auto","RGF","Bagging","Voting","Stacking"]



start_time = time.time()

clf = custom_RF.fit(x_train, y_train)
y_pred_train = clf.predict(x_train)

execution_time_calculator(start_time)


metrics_calculator(y_pred_train,y_train,"train")

--- 0.14200186729431152 seconds ---
******** train ******** :
F1:                  0.807631160572337
Accuracy:            0.8641975308641975
Precision:           0.7426900584795322
Recall:              0.8850174216027874
AP:                  0.694330677642616
RoC_AuC:             0.8696610286821884
MSE:                 0.13580246913580246
RMSE:                0.3685138655950444
              precision    recall  f1-score   support

         0.0       0.94      0.85      0.90       604
         1.0       0.74      0.89      0.81       287

    accuracy                           0.86       891
   macro avg       0.84      0.87      0.85       891
weighted avg       0.88      0.86      0.87       891



In [40]:
# classifiers = all_models_test(df,models,models_name,x_train,y_train)

## Load test data

In [41]:
df_test = pd.read_csv("./data/test.csv")
ids_col = df_test["PassengerId"].values
df_test,_ = df_clean(df_test,1,age_mean)
df_test,_,_,_ = df_encoder(df_test,1,[LE_Sex, LE_Cabin, LE_Embarked])

array_test = df_test.values
x_test = array_test[:,]

In [42]:
# df_test.info()
# df_test

In [43]:
# y_pred_test = classifiers[8].predict(x_test)
y_pred_test = clf.predict(x_test)

In [44]:
# y_pred_test

In [45]:
df_result = pd.DataFrame(list(zip(list(ids_col),list(y_pred_test))), columns =['PassengerId','Survived'])
df_result['Survived'] = df_result['Survived'].astype('int64')

In [46]:
df_result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [47]:
# compression_opts = dict(method='zip',archive_name='out.csv')  

df_result.to_csv('out.csv', index=False)  